In [1]:
import pandas as pd
import numpy as np
import selenium
from selenium import webdriver
import chromedriver_autoinstaller
import requests
import time
import re

In [2]:
class Driver:
    @staticmethod
    def driver_init():
        chromedriver_autoinstaller.install()
        options = webdriver.ChromeOptions()
        #options.add_argument('headless') # jupyter notebook 환경에서 주석처리
        options.add_argument('lang=ko_KR')
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36")
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-notifications')
        options.add_argument('--disable-popup-blocking')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-browser-side-navigation')
        options.add_argument('disable-gpu')

        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(10)
        driver.set_window_size(1600, 1100)

        return driver

In [3]:
driver = Driver.driver_init()
article_list = []
driver.get('https://ustr.gov/about-us/policy-offices/press-office/press-releases/2023')
while True:
    max_height = driver.execute_script('return document.body.scrollHeight')
    max_try = max_height//900
    y_location = 0
    for m in range(max_try):
        y_location+=900
        time.sleep(1)
        driver.execute_script(f"window.scrollTo(0, {y_location})")
    articles = driver.find_elements('xpath', '//ul[@class="listing"]/li/a')
    article_urls = [i.get_attribute('href') for i in articles]
    article_list.extend(article_urls)
    break

In [4]:
len(article_list)

305

In [5]:
for idx, article in enumerate(article_list):
    driver.get(article)

    date = driver.find_element('xpath', '//div[@class="col-sm-12 col-lg-9 column content-column"]/p').get_attribute('innerText')
    title = driver.find_element('xpath', '//h1[@class="pageTitle"]/span').get_attribute('innerText')
    main = driver.find_element('xpath', '//article[@role="article"]').get_attribute('innerText')

    file_name = f'USTR_press_release_{idx}.txt'

    with open(file_name, 'w', encoding = 'utf-8') as file:
        file.write('<Date>' + '\n')
        file.write(date + '\n')
        file.write('<Title>' + '\n')
        file.write(title + '\n')
        file.write('<Content>' + '\n')
        #file.write(header_desc + '\n')
        file.write(main)